In [1]:
import os
import folium
from folium import FeatureGroup, LayerControl, Map, Marker


import time
import selenium
from selenium import webdriver
import datetime


import re
import pandas as pd 

CSV_EXTENSION  = 'csv'
EXCEL_EXTENSION = 'xls'

datasheets = ["all-ewb","ghana.csv", "morocco.csv", "uganda.csv"]

class EWBMap():
    """
    Object to reprent map
    Attributes: 
    - df: Dataframe with data
    """

    def __init__(self):
        df = None

#####
class EWBLayer():
    def __init__(self):
        name = ""
        points = []
        
class EWBPoint():
    def __init__(self):
        name = ""
        lat = 0
        long = 0
        description = ""
#####     
        
        
def determine_extension(filename: str) -> str:
    """
    Returns type of extension constant  associated with a Filename.
    If Extension is not supperted IOError is raised.
    """
    name_pattern = r'[\w\s_]+'
    excel_reg = re.compile(name_pattern + r'.(xls|xlsx|xlsm|xlsb|xls)$')
    csv_reg = re.compile(name_pattern + r'.csv$')
    extension = None
        
    if excel_reg.match(filename):
        extension = EXCEL_EXTENSION
    elif csv_reg.match(filename):
        extension = CSV_EXTENSION
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    
    return extension

    
def get_dataframe(filename: str) -> pd.DataFrame:
    """
    Returns a dataframe from an excel or csv file
    """
    
    extension = determine_extension(filename)
    df = None
    if extension == EXCEL_EXTENSION:
        df = pd.read_excel(filename)
    elif extension == CSV_EXTENSION:
        df = pd.read_csv(filename)
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    
    return df


def get_map(filename: str) -> EWBMap:
    """"
    Returns a map object read from a csv or excel file
    """
    
    map_object = EWBMap()
    map_object.df = get_dataframe(filename)
    return map_object

#####
def get_layer(filename: str) -> EWBLayer:
    layer_object = EWBLayer()
    layer_object.name = filename
    map_df = get_dataframe(filename)
    for row in map_df.itertuples():
        point = EWBPoint()
        point.name = row.name
        point.lat = row.latitude
        point.long = row.longitude
        point.description = row.description
        
        layer_object.points.append(point)
#####   
    
def make_popups(layer, lat, lon, title, date, description, icon, color):
    folium.Marker(
        location = [lat, lon],
        icon = folium.Icon(icon=icon, color=color, prefix='fa'),
        tooltip = title,
        popup = folium.Popup(
            folium.Html('<b>%s</b> <br> <i>%s</i> <br> %s' %(title, date, description), script=True),
            min_width=100,
            max_width=450
            )
    ).add_to(layer)
    
    
    
# pass in the map to re-center around fixed point
def recenter(map, string, ptsList=[]):
    # center around the town
    if string == 'town':
        x = 45.5236
        y = -122.6750
        map.location = [x, y]
    # ghana: show all datapoints in EWB-Ghana
    # all-ewb: show transportation and overview of all EWB project location
    elif string == 'ghana' or string == 'all-ewb':
        # if input location list is empty
        if len(ptsList)<1:
            print("ERROR: list input is empty")
        else:
            # find the SW and NE bound of the dataset
            first = ptsList[0]
            xMin = first[0]
            yMin = first[1]
            xMax = xMin
            yMax = yMin
            for pt in ptsList:
                xMin = min(xMin, pt[0])
                xMax = max(xMax, pt[0])
                yMin = min(yMin, pt[1])
                yMax = max(yMax, pt[1])
            # adjust the map bound according the upper and lower bound of the dataset
            map.fit_bounds([[xMin, yMin], [xMax, yMax]])
    # if the input string does not match any of the options
    else:
        print("ERROR: wrong string input")
    return map


def map_to_html (m, map_name, file_path =""):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    path = file_path+map_name+".html"
    m.save(path)
    return path

def map_to_png(m, map_name, file_path ="", browser= 'Safari'):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    fn = map_name+".html"
    m.save(fn)
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    delay =5
    if browser == 'Safari':
        browser = webdriver.Safari()
    elif browser == 'Firefox':
        browser = webdriver.Firefox()
    else:
        browser = webdriver.Chrome()
    
    browser.get(tmpurl)
    time.sleep(delay)
    browser.save_screenshot(file_path+map_name+'.png')
    browser.quit()



ModuleNotFoundError: No module named 'selenium'

In [8]:
map_layers = []

for filename in datasheets:
    map_layers.append(get_layer(filename))
    

m = Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)


for layer in map_layers:
    feature_group = FeatureGroup(name=layer.name)
    for point in layer.points:
        date = datetime.datetime.now()
        color='lightgray'
        icon='home'
        make_popups(feature_group, point.lat, point.lon, point.name, date, point.description, icon, color)
        
    feature_group.add_to(m)

LayerControl.add_to(m)

m

#m.save(os.path.join('results', 'FeatureGroup.html'))

In [ ]:
map_to_html(m, "EWB Map", "")